# Settings

In [50]:
import os
import sys
import time
import numpy as np

import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

from PIL import Image, ImageDraw

In [6]:
import utils
from vgg16 import Vgg16

# Hyperparameters

In [32]:
list_content_weight = [1000,750,500,250,100,75,50,25,10,7.5, 5, 2.5,1]

In [33]:
iters = 1000
content_image_path = '../../test-images/cat.jpg'
style_image_path = '../../test-images/the_wave.jpg'
style_size = 512
content_size = 512
output_image_path = '../../test-images/'
vgg_model_dir = 'models/'
cuda = 1
style_weight = 5.0
content_weight = 1000.0
learning_rate = 1e1
log_interval = 50

# Optimizer

## Make gif

In [48]:
def tensor_to_numpy(tensor, cuda=False):
    if cuda:
        img = tensor.clone().cpu().clamp(0, 255).numpy()
    else:
        img = tensor.clone().clamp(0, 255).numpy()
    img = img.transpose(1, 2, 0).astype('uint8')
    img = Image.fromarray(img)
    return(img)
    #img.save(filename)

In [51]:
pictures = []

content_image = utils.tensor_load_rgbimage(content_image_path, size=content_size, keep_asp=True)
content_image = content_image.unsqueeze(0)
content_image = Variable(utils.preprocess_batch(content_image), requires_grad=False)
utils.subtract_imagenet_mean_batch(content_image)
style_image = utils.tensor_load_rgbimage(style_image_path, size=style_size)
style_image = style_image.unsqueeze(0)
style_image = Variable(utils.preprocess_batch(style_image), requires_grad=False)
utils.subtract_imagenet_mean_batch(style_image)

vgg = Vgg16()
utils.init_vgg16(vgg_model_dir)
vgg.load_state_dict(torch.load(os.path.join(vgg_model_dir, "vgg16.weight")))

if cuda:
    content_image = content_image.cuda()
    style_image = style_image.cuda()
    vgg.cuda()

for i, content_weight in enumerate(list_content_weight):
    features_content = vgg(content_image)
    f_xc_c = Variable(features_content[1].data, requires_grad=False)
    features_style = vgg(style_image)
    gram_style = [utils.gram_matrix(y) for y in features_style]
    # init optimizer
    output = Variable(content_image.data, requires_grad=True)
    optimizer = Adam([output], lr=learning_rate)
    mse_loss = torch.nn.MSELoss()

    #modify the feature map using gain
    f_xs_s = Variable(features_content[1].data, requires_grad=False)
    g_min = 0.7
    g_max = 5
    gain = f_xs_s / (f_xc_c + 10e-4)
    gain_clamped = torch.clamp(gain, min=g_min, max=g_max)
    f_modified = f_xc_c * gain

    # optimizing the images
    for e in range(iters):
        utils.add_imagenet_mean_batch(output)
        output.data.clamp_(0, 255)	
        utils.subtract_imagenet_mean_batch(output)

        optimizer.zero_grad()
        features_y = vgg(output)
        content_loss = content_weight * mse_loss(features_y[1], f_modified)

        style_loss = 0.
        for m in range(len(features_y)):
            gram_y = utils.gram_matrix(features_y[m])
            gram_s = Variable(gram_style[m].data, requires_grad=False)
            style_loss += style_weight * mse_loss(gram_y, gram_s)

        total_loss = content_loss + style_loss

        if (e + 1) % log_interval == 0:
            #print(total_loss.data.cpu().numpy()[0])
            print('iterations:',e)
            print('loss:',total_loss.data.cpu().numpy())
        total_loss.backward()
    
        optimizer.step()
    # save the image
    pictures.append(tensor_to_numpy(output.data[0], cuda))
    utils.tensor_save_bgrimage(output.data[0], output_image_path + 'output_' + str(i) + '.jpg', cuda)

In [55]:
pictures[0].save('cat.gif', format='GIF', append_images=pictures[1:], save_all=True, duration=100, loop=0)

## Single optimizer

In [19]:
content_image = utils.tensor_load_rgbimage(content_image_path, size=content_size, keep_asp=True)
content_image = content_image.unsqueeze(0)
content_image = Variable(utils.preprocess_batch(content_image), requires_grad=False)
utils.subtract_imagenet_mean_batch(content_image)
style_image = utils.tensor_load_rgbimage(style_image_path, size=style_size)
style_image = style_image.unsqueeze(0)
style_image = Variable(utils.preprocess_batch(style_image), requires_grad=False)
utils.subtract_imagenet_mean_batch(style_image)

vgg = Vgg16()
utils.init_vgg16(vgg_model_dir)
vgg.load_state_dict(torch.load(os.path.join(vgg_model_dir, "vgg16.weight")))

if cuda:
    content_image = content_image.cuda()
    style_image = style_image.cuda()
    vgg.cuda()
features_content = vgg(content_image)
f_xc_c = Variable(features_content[1].data, requires_grad=False)
features_style = vgg(style_image)
gram_style = [utils.gram_matrix(y) for y in features_style]
# init optimizer
output = Variable(content_image.data, requires_grad=True)
optimizer = Adam([output], lr=learning_rate)
mse_loss = torch.nn.MSELoss()

#modify the feature map using gain
f_xs_s = Variable(features_content[1].data, requires_grad=False)
g_min = 0.7
g_max = 5
gain = f_xs_s / (f_xc_c + 10e-4)
gain_clamped = torch.clamp(gain, min=g_min, max=g_max)
f_modified = f_xc_c * gain

# optimizing the images
for e in range(iters):
    utils.add_imagenet_mean_batch(output)
    output.data.clamp_(0, 255)	
    utils.subtract_imagenet_mean_batch(output)

    optimizer.zero_grad()
    features_y = vgg(output)
    content_loss = content_weight * mse_loss(features_y[1], f_modified)

    style_loss = 0.
    for m in range(len(features_y)):
        gram_y = utils.gram_matrix(features_y[m])
        gram_s = Variable(gram_style[m].data, requires_grad=False)
        style_loss += style_weight * mse_loss(gram_y, gram_s)

    total_loss = content_loss + style_loss

    if (e + 1) % log_interval == 0:
        #print(total_loss.data.cpu().numpy()[0])
        print('iterations:',e)
        print('loss:',total_loss.data.cpu().numpy())
    total_loss.backward()
    
    optimizer.step()
# save the image	
utils.tensor_save_bgrimage(output.data[0], output_image_path, cuda)

iterations: 49
loss: 1356320.8
iterations: 99
loss: 1014319.3
iterations: 149
loss: 936335.75
iterations: 199
loss: 853631.2
iterations: 249
loss: 918182.9
iterations: 299
loss: 1024799.9
iterations: 349
loss: 1222416.2
iterations: 399
loss: 854556.9
iterations: 449
loss: 1034787.8
iterations: 499
loss: 1265034.2
iterations: 549
loss: 1129760.5
iterations: 599
loss: 1504305.2
iterations: 649
loss: 807273.25
iterations: 699
loss: 917512.1
iterations: 749
loss: 1601480.0
iterations: 799
loss: 804395.25
iterations: 849
loss: 975821.9
iterations: 899
loss: 869358.06
iterations: 949
loss: 942463.5
iterations: 999
loss: 969251.9
